In [1]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ], shape=(11314,))

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 8809, 1825, 1828], shape=(11314,))

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


## Solución Desafío 1

### Parte 1: Similaridad de 5 documentos al azar


In [22]:
import random

# Fijamos semilla para reproducibilidad
np.random.seed(10)

# Seleccionamos 5 documentos al azar
random_docs = np.random.choice(len(newsgroups_train.data), 5, replace=False)
print("Documentos:", random_docs)
print("Clases:", [newsgroups_train.target_names[y_train[i]] for i in random_docs])


Documentos: [10809  1679  3731 10076  8799]
Clases: ['sci.space', 'sci.crypt', 'alt.atheism', 'talk.religion.misc', 'talk.politics.mideast']


In [23]:
def analizar_documento_similar(doc_idx, X_train, y_train, newsgroups_train, top_n=5):
    
    cossim = cosine_similarity(X_train[doc_idx], X_train)[0]
    
    most_similar_idx = np.argsort(cossim)[::-1][1:top_n+1]
    
    print(f"\nDOCUMENTO {doc_idx}")
    print(f"Clase: {newsgroups_train.target_names[y_train[doc_idx]]}")
    print(f"Contenido:")
    print(newsgroups_train.data[doc_idx])
    
    print(f"\nLos {top_n} documentos mas similares:")
    for i, sim_idx in enumerate(most_similar_idx):
        print(f"\n{i+1}. Documento {sim_idx} (similaridad: {cossim[sim_idx]:.4f})")
        print(f"   Clase: {newsgroups_train.target_names[y_train[sim_idx]]}")
        print(f"   Contenido: {newsgroups_train.data[sim_idx]}")
    
    clases_similares = [newsgroups_train.target_names[y_train[idx]] for idx in most_similar_idx]
    clase_original = newsgroups_train.target_names[y_train[doc_idx]]
    coincidencias = sum(1 for clase in clases_similares if clase == clase_original)
    
    print(f"\nAnalisis: {coincidencias}/{top_n} documentos similares pertenecen a la misma clase '{clase_original}'")
    
    return most_similar_idx, cossim[most_similar_idx]


In [24]:
analizar_documento_similar(random_docs[0], X_train, y_train, newsgroups_train)



DOCUMENTO 10809
Clase: sci.space
Contenido:
a

What about positional uncertainties in S-L 1993e?   I assume we know where
and what Galileo is doing within a few meters.   But without the
HGA,  don't we have to have some pretty good ideas, of where to look
before imaging?  If the HGA was working,  they could slew around
in near real time (Less speed of light delay).  But when they were
imaging toutatis????  didn't someone have to get lucky on a guess to
find the first images?   

Also, I imagine S-L 1993e will be mostly a visual image.  so how will
that affect the other imaging missions.  with the LGA,  there is a real
tight allocation of bandwidth.   It may be premature to hope for answers,
but I thought i'd throw it on the floor.

Los 5 documentos mas similares:

1. Documento 1483 (similaridad: 0.2893)
   Clase: sci.space
   Contenido: INteresting question about Galileo.

Galileo's HGA  is stuck.   

The HGA was left closed,  because galileo  had a venus flyby.

If the HGA were point

(array([1483, 9623, 3222, 6437, 6894]),
 array([0.28929452, 0.23570679, 0.22455096, 0.22368227, 0.22020699]))

### Comentarios
Creo que la razón principal por la cual solo dos de los cinco documentos pertenecen a la misma clase es poruq la similaridad coseno no captura la semántica si no que al compartir vocabulario común y no tener la misma relación temática real (ciencia vs violaciones de ddhh) cae igual.


## Parte 2: Optimización de modelos Naïve Bayes


In [25]:
def evaluar_modelo(classifier, X_train, y_train, X_test, y_test, nombre):
    
    classifier.fit(X_train, y_train)
    
    y_pred = classifier.predict(X_test)
    
    f1 = f1_score(y_test, y_pred, average='macro')
    
    print(f"{nombre}: F1-score macro = {f1:.4f}")
    
    return f1

vectorizer_configs = [
    ("TfidfVectorizer default", TfidfVectorizer()),
    ("TfidfVectorizer + min_df=2", TfidfVectorizer(min_df=2)),
    ("TfidfVectorizer + max_df=0.8", TfidfVectorizer(max_df=0.8)),
    ("TfidfVectorizer + min_df=2, max_df=0.8", TfidfVectorizer(min_df=2, max_df=0.8)),
    ("TfidfVectorizer + ngram=(1,2)", TfidfVectorizer(ngram_range=(1, 2))),
    ("TfidfVectorizer + ngram=(1,2) + min_df=2", TfidfVectorizer(ngram_range=(1, 2), min_df=2)),
    ("TfidfVectorizer + max_features=10000", TfidfVectorizer(max_features=10000)),
    ("CountVectorizer default", CountVectorizer()),
    ("CountVectorizer + min_df=2", CountVectorizer(min_df=2)),
]

models = [
    ("MultinomialNB default", MultinomialNB()),
    ("MultinomialNB alpha=0.1", MultinomialNB(alpha=0.1)),
    ("MultinomialNB alpha=0.5", MultinomialNB(alpha=0.5)),
    ("MultinomialNB alpha=2.0", MultinomialNB(alpha=2.0)),
    ("ComplementNB default", ComplementNB()),
    ("ComplementNB alpha=0.1", ComplementNB(alpha=0.1)),
    ("ComplementNB alpha=0.5", ComplementNB(alpha=0.5)),
]


In [26]:
resultados = []

for vec_name, vectorizer in vectorizer_configs:
    print(f"\nProbando {vec_name}")
    
    X_train_vec = vectorizer.fit_transform(newsgroups_train.data)
    X_test_vec = vectorizer.transform(newsgroups_test.data)
    
    for model_name, model in models:
        from copy import deepcopy
        model_copy = deepcopy(model)
        
        nombre_completo = f"{vec_name} + {model_name}"
        f1 = evaluar_modelo(model_copy, X_train_vec, y_train, X_test_vec, y_test, nombre_completo)
        
        resultados.append((nombre_completo, f1, vec_name, model_name))

resultados.sort(key=lambda x: x[1], reverse=True)

print("--------------------------------")
print("MEJORES RESULTADOS:")
print("--------------------------------")
for i, (nombre, f1, vec_name, model_name) in enumerate(resultados[:10]):
    print(f"{i+1}. {nombre}: F1-score = {f1:.4f}")

mejor_resultado = resultados[0]
print(f"\nMEJOR CONFIGURACION: {mejor_resultado[0]} con F1-score = {mejor_resultado[1]:.4f}")



Probando TfidfVectorizer default
TfidfVectorizer default + MultinomialNB default: F1-score macro = 0.5854
TfidfVectorizer default + MultinomialNB alpha=0.1: F1-score macro = 0.6565
TfidfVectorizer default + MultinomialNB alpha=0.5: F1-score macro = 0.6153
TfidfVectorizer default + MultinomialNB alpha=2.0: F1-score macro = 0.5400
TfidfVectorizer default + ComplementNB default: F1-score macro = 0.6930
TfidfVectorizer default + ComplementNB alpha=0.1: F1-score macro = 0.6954
TfidfVectorizer default + ComplementNB alpha=0.5: F1-score macro = 0.6961

Probando TfidfVectorizer + min_df=2
TfidfVectorizer + min_df=2 + MultinomialNB default: F1-score macro = 0.5970
TfidfVectorizer + min_df=2 + MultinomialNB alpha=0.1: F1-score macro = 0.6727
TfidfVectorizer + min_df=2 + MultinomialNB alpha=0.5: F1-score macro = 0.6289
TfidfVectorizer + min_df=2 + MultinomialNB alpha=2.0: F1-score macro = 0.5563
TfidfVectorizer + min_df=2 + ComplementNB default: F1-score macro = 0.6935
TfidfVectorizer + min_df=2

In [27]:
X_train_T = X_train.T

print(f"Matriz original (documento-termino): {X_train.shape}")
print(f"Matriz transpuesta (termino-documento): {X_train_T.shape}")

palabras_interes = ['science', 'space', 'war', 'medical', 'military']

palabras_validas = []
indices_palabras = []

for palabra in palabras_interes:
    if palabra in tfidfvect.vocabulary_:
        idx = tfidfvect.vocabulary_[palabra]
        palabras_validas.append(palabra)
        indices_palabras.append(idx)
        print(f"'{palabra}' encontrada en indice {idx}")
    else:
        print(f"'{palabra}' NO encontrada en vocabulario")

print(f"\nPalabras validas para analizar: {palabras_validas}")


Matriz original (documento-termino): (11314, 101631)
Matriz transpuesta (termino-documento): (101631, 11314)
'science' encontrada en indice 80937
'space' encontrada en indice 84097
'war' encontrada en indice 95770
'medical' encontrada en indice 60703
'military' encontrada en indice 61759

Palabras validas para analizar: ['science', 'space', 'war', 'medical', 'military']


In [28]:
def analizar_palabra_similar(palabra_idx, palabra, X_train_T, idx2word):
    word_vector = X_train_T[palabra_idx]
    cossim = cosine_similarity(word_vector, X_train_T)[0]
    
    most_similar_idx = np.argsort(cossim)[::-1][1:5+1]
    
    print(f"\nPALABRA: '{palabra}' (indice {palabra_idx})")
    print(f"Las {5} palabras mas similares:")
    
    for i, sim_idx in enumerate(most_similar_idx):
        palabra_similar = idx2word[sim_idx]
        similaridad = cossim[sim_idx]
        print(f"{i+1}. '{palabra_similar}' (similaridad: {similaridad:.4f})")
    
    return most_similar_idx, cossim[most_similar_idx]

for i, (palabra, idx) in enumerate(zip(palabras_validas, indices_palabras)):
    analizar_palabra_similar(idx, palabra, X_train_T, idx2word)



PALABRA: 'science' (indice 80937)
Las 5 palabras mas similares:
1. 'cognitivists' (similaridad: 0.3948)
2. 'behaviorists' (similaridad: 0.3948)
3. 'scientific' (similaridad: 0.3466)
4. 'empirical' (similaridad: 0.2810)
5. 'sects' (similaridad: 0.2622)

PALABRA: 'space' (indice 84097)
Las 5 palabras mas similares:
1. 'nasa' (similaridad: 0.3304)
2. 'seds' (similaridad: 0.2966)
3. 'shuttle' (similaridad: 0.2928)
4. 'enfant' (similaridad: 0.2803)
5. 'seti' (similaridad: 0.2465)

PALABRA: 'war' (indice 95770)
Las 5 palabras mas similares:
1. 'irag' (similaridad: 0.2548)
2. 'dresden' (similaridad: 0.2376)
3. '1948' (similaridad: 0.2366)
4. 'lauches' (similaridad: 0.2071)
5. 'drugs' (similaridad: 0.1981)

PALABRA: 'medical' (indice 60703)
Las 5 palabras mas similares:
1. 'romano' (similaridad: 0.2823)
2. 'hospitals' (similaridad: 0.2751)
3. 'recuperation' (similaridad: 0.2682)
4. 'providers' (similaridad: 0.2391)
5. 'relelvant' (similaridad: 0.2278)

PALABRA: 'military' (indice 61759)
Las 5